In [1]:
# script to replace the base mechanism with new reactions
# Includes every reaction and species calculated above the uncertainty threshold

In [2]:
import os
import pickle
import subprocess
import numpy as np

import rmgpy.chemkin
import rmgpy.data.kinetics


import matplotlib.pyplot as plt
%matplotlib inline

In [4]:
# Load the base model
basedir = '/work/westgroup/harris.se/autoscience/autoscience/butane/models/rmg_model'
# basedir = '/work/westgroup/harris.se/autoscience/autoscience/butane/sensitivity_analysis/base_rmg24'
# basedir = '/work/westgroup/harris.se/autoscience/autoscience/butane/models/rmg_small_core'
# new_model_dir = '/work/westgroup/harris.se/autoscience/autoscience/butane/improved_models'

# basedir = '/work/westgroup/harris.se/autoscience/autoscience/debug/s787_rxns/'
new_model_dir = basedir
base_chemkin = os.path.join(basedir, 'chem_annotated.inp')
dictionary = os.path.join(basedir, 'species_dictionary.txt')
transport = os.path.join(basedir, 'tran.dat')
species_list, reaction_list = rmgpy.chemkin.load_chemkin_file(base_chemkin, dictionary_path=dictionary, transport_path=transport)
print(f'{len(species_list)} species, {len(reaction_list)} reactions')


110 species, 1822 reactions


In [5]:
# Load the new kinetics library
DFT_DIR = "/work/westgroup/harris.se/autoscience/autoscience/butane/dft/"
kinetics_lib = os.path.join(DFT_DIR, 'kinetics', 'kinetics')
ark_kinetics_database = rmgpy.data.kinetics.KineticsDatabase()
ark_kinetics_database.load_libraries(kinetics_lib)
print(f'{len(ark_kinetics_database.libraries[""].entries)} entries loaded')

105 entries loaded


In [6]:
# load the uncertainty matrix
covariance_file = '/work/westgroup/harris.se/autoscience/autoscience/uncertainty/butane_covariance.pickle'
with open(covariance_file, 'rb') as handle:
    Sigma_k = pickle.load(handle)

uncertainty_cutoff = 3.0
# uncertainty_cutoff = 0.5
# uncertainty_cutoff = 0.0

change_list = []
for i in range(0, len(reaction_list)):
    if Sigma_k[i, i] > uncertainty_cutoff:
        change_list.append(i)



In [7]:
total_changed = 0
for key in ark_kinetics_database.libraries[''].entries.keys():
    entry = ark_kinetics_database.libraries[''].entries[key]
    index = entry.index
#     print(index, '\t', entry.label, '\t', reaction_list[index])
    for i in range(0, len(reaction_list)):
        if entry.item.is_isomorphic(reaction_list[i]):
            rmg_rxn_index = i
            break
    else:  
        print(f'could not match reaction index {index}')
        continue
  
    
#     # change every possible reaction
#     print(f'changing reaction {rmg_rxn_index} {reaction_list[rmg_rxn_index]}')
#     reaction_list[rmg_rxn_index].kinetics = entry.data
    
    if rmg_rxn_index in change_list:
        print(f'changing reaction {rmg_rxn_index} {reaction_list[rmg_rxn_index]}')
        reaction_list[rmg_rxn_index].kinetics = entry.data
        total_changed += 1

#     if rmg_rxn_index in change_list or rmg_rxn_index in change_list_fs:
#         print(f'changing reaction {rmg_rxn_index} {reaction_list[rmg_rxn_index]}')
#         reaction_list[rmg_rxn_index].kinetics = entry.data
#         total_changed += 1
print(total_changed, 'changed')

changing reaction 213 HO2(16) + C2H5(33) <=> H2O2(17) + C2H4(11)
changing reaction 324 C4H8OOH2-4O2(229) <=> C4H8OOH1-3O2(225)
changing reaction 370 CH3CHO(35) + SC4H9(183) <=> CH2CHO(21) + butane(1)
changing reaction 371 CH3CHO(35) + PC4H9(182) <=> CH2CHO(21) + butane(1)
changing reaction 418 HO2(16) + CH3CO(20) <=> H2O2(17) + CH2CO(24)
changing reaction 419 HO2(16) + CH2CHO(21) <=> H2O2(17) + CH2CO(24)
changing reaction 420 HO2(16) + SC4H9(183) <=> H2O2(17) + C4H8-2(189)
changing reaction 427 CH2CHO(21) + CH2CHO(21) <=> CH2CO(24) + CH3CHO(35)
changing reaction 574 CH3O(31) + C3H5-A(94) <=> CH2O(9) + C3H6(12)
changing reaction 581 CH2CHO(21) + C3H5-A(94) <=> CH2CO(24) + C3H6(12)
changing reaction 720 CH2CHO(21) + C4H71-3(190) <=> CH2CO(24) + C4H8-2(189)
changing reaction 722 CH2CHO(21) + C4H71-3(190) <=> CH2CO(24) + C4H8-1(188)
changing reaction 725 CH3O(31) + C4H71-3(190) <=> CH2O(9) + C4H8-2(189)
changing reaction 739 CH2CHO(21) + C4H8-1(188) <=> CH3CHO(35) + C4H71-3(190)
changing r

# 

In [8]:
# Load the new thermo library
thermo_lib = os.path.join(DFT_DIR, 'thermo', 'thermo')
ark_thermo_database = rmgpy.data.thermo.ThermoDatabase()
ark_thermo_database.load_libraries(thermo_lib)
print(f'{len(ark_thermo_database.libraries["harris_butane"].entries)} entries')

110 entries


In [9]:
# stitch each of the library thermos into the mechanism

plot_index = [96]
thermo_change_list = [87, 90, 109, 99, 98, 80, 100, 97, 81, 91, 82, 93]  # 3.0 cutoff
thermo_change_list_fs = [97, 98, 99, 81, 100, 87, 109, 80, 90, 93, 91, 82]  # 3.0 cutoff, flame speeds these are the same

# thermo_change_list = [63, 4, 66, 57, 61, 97, 62, 59, 83, 64, 74, 60, 98, 39, 45, 37, 47, 70, 73, 38, 72,  # 1.0 cutoff, large molecules only
#                       58, 85, 69, 52, 65, 56, 99, 81, 92, 67, 51, 101, 68, 95, 54, 100, 105, 103, 42, 87,
#                       55, 43, 84, 46, 53, 109, 104, 107, 77, 102, 78, 71, 88, 76, 90, 93, 44, 89, 86, 108, 106, 82, 75]

# thermo_change_list = [97, 83, 98, 85, 99, 81, 92, 101, 100, 105, 103, 87, 84, 109, 107, 102, 88, 90, 93, 89, 86, 108, 106, 82]  # 1.5 cutoff, MW>48
# thermo_change_list = [97, 98, 85, 99, 81, 101, 100, 105, 87, 109, 90, 93, 82]  # 2.5 cutoff, large molecules only


# thermo_change_list = [85,  # 2.0 cutoff
#  87, 84, 105, 103, 90, 102, 109, 99, 107, 83, 92, 98, 80, 100, 97, 88, 81,
#  101, 106, 86, 108, 91, 82, 96, 93, 89]
sp_changed = 0
for key in ark_thermo_database.libraries['harris_butane'].entries.keys():
    entry = ark_thermo_database.libraries['harris_butane'].entries[key]
#     print(entry)

    for i, species in enumerate(species_list):
#         if entry.item.is_isomorphic(species.molecule[0]):
        if entry.item.smiles == species.smiles:
            rmg_species_index = i
            break
    else:
        entry_sp = rmgpy.species.Species(smiles=entry.item.smiles)
        for j, species in enumerate(species_list):
            if entry_sp.is_isomorphic(species.molecule[0]):
                rmg_species_index = i
                'matched'
                break
        else:
#             print(f'could not match species index {entry}')
            continue
            raise ValueError (f'could not match species index {entry}')

#     # only change GAV entries
#     if 'library' not in species_list[rmg_species_index].thermo.comment.lower():
#         print(f'Changing estimated thermo {rmg_species_index}: {species_list[rmg_species_index]}')
#         try:
#             display(species_list[rmg_species_index])
#         except:
#             pass
#         plot_thermos([species_list[rmg_species_index], entry.data], ['RMG', 'DFT'])
#         species_list[rmg_species_index].thermo = entry.data
#         print(rmg_species_index)
#     if rmg_species_index in thermo_change_list:
#         print(f'Changing estimated thermo {rmg_species_index}: {species_list[rmg_species_index]}')
#         species_list[rmg_species_index].thermo = entry.data
        
        
    if rmg_species_index in thermo_change_list or rmg_species_index in thermo_change_list_fs:
        print(f'Changing estimated thermo {rmg_species_index}: {species_list[rmg_species_index]}')
        species_list[rmg_species_index].thermo = entry.data
        
        sp_changed += 1
    
print(sp_changed, 'changed')   
        
#     # change everything
#     print(f'Changing estimated thermo {rmg_species_index}: {species_list[rmg_species_index]}')
#     if rmg_species_index == 85:
# #         plot_thermos([species_list[rmg_species_index], entry.data], ['RMG', 'DFT'])
#         species_list[rmg_species_index].thermo = entry.data

    
    # only change GAV
    
    
#     if rmg_species_index in plot_index:
#         plot_thermos([species_list[rmg_species_index], entry.data], ['RMG', 'DFT'])
#         print(f'Changing estimated thermo {rmg_species_index}: {species_list[rmg_species_index]}')
#         species_list[rmg_species_index].thermo = entry.data

#     break
#     print(species_list[rmg_species_index])
#     print(f'Changing reaction {rmg_rxn_index}')
#     if rmg_rxn_index == 288:
#         reaction_list[rmg_rxn_index].kinetics = entry.data
# #     break
#     # compare the kinetics
#     plot_kinetics([reaction_list[rmg_rxn_index], entry.data], ['RMG', 'DFT'])
    

Changing estimated thermo 80: [CH2]C[O](703)
Changing estimated thermo 81: [CH2]C[CH]C(748)
Changing estimated thermo 82: [CH2]CC(C)[O](766)
Changing estimated thermo 87: [CH2]C(=O)COO(911)
Changing estimated thermo 90: [O]OCC(=O)COO(928)
Changing estimated thermo 91: [O]C[O](988)
Changing estimated thermo 93: [CH2]C=C[CH2](2534)
Changing estimated thermo 97: C=CC(C)O[O](4243)
Changing estimated thermo 98: CC=CCO[O](4244)
Changing estimated thermo 99: [CH2]C1CC1(4343)
Changing estimated thermo 100: C=CCCO[O](4407)
Changing estimated thermo 109: [O]OC1CCOOC1(9510)
12 changed


In [10]:
# Save the new mechanism
chemkin_file = os.path.join('/work/westgroup/harris.se/autoscience/autoscience/butane/models/modifications', f'cutoff3_20221110.inp')
rmgpy.chemkin.save_chemkin_file(chemkin_file, species_list, reaction_list, verbose=True, check_for_duplicates=True)
subprocess.run(['ck2cti', f'--input={chemkin_file}', f'--transport={transport}', f'--output={chemkin_file[:-4]}.cti'])
# os.remove(chemkin_file)


Wrote CTI mechanism file to '/work/westgroup/harris.se/autoscience/autoscience/butane/models/modifications/cutoff3_20221110.cti'.
Mechanism contains 110 species and 1850 reactions.
Validating mechanism...PASSED.


CompletedProcess(args=['ck2cti', '--input=/work/westgroup/harris.se/autoscience/autoscience/butane/models/modifications/cutoff3_20221110.inp', '--transport=/work/westgroup/harris.se/autoscience/autoscience/butane/models/rmg_model/tran.dat', '--output=/work/westgroup/harris.se/autoscience/autoscience/butane/models/modifications/cutoff3_20221110.cti'], returncode=0)